## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-11-13-06-56 +0000,nypost,Bravo star Jennifer Welch slammed for claiming...,https://nypost.com/2025/08/11/us-news/bravo-st...
1,2025-08-11-13-02-12 +0000,nyt,Hundreds of Indian Lawmakers Detained at a Pro...,https://www.nytimes.com/2025/08/11/world/asia/...
2,2025-08-11-13-01-48 +0000,nyt,Paramount Strikes Seven-Year Deal to Stream UF...,https://www.nytimes.com/2025/08/11/business/pa...
3,2025-08-11-13-00-00 +0000,wsj,"Stock Market Today: Nvidia, AMD Stocks Slip; D...",https://www.wsj.com/livecoverage/stock-market-...
4,2025-08-11-12-55-07 +0000,wapo,Colombian presidential hopeful dies after bein...,https://www.washingtonpost.com/world/2025/08/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2298/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,35
370,putin,12
58,china,11
170,gaza,10
239,new,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
147,2025-08-10-20-57-00 +0000,wsj,Intel CEO Lip-Bu Tan is set to visit the White...,https://www.wsj.com/tech/intel-ceo-singled-out...,81
49,2025-08-11-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Trump-Putin summit ...,https://nypost.com/2025/08/11/us-news/ny-postc...,80
117,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...,80
184,2025-08-10-16-46-37 +0000,nyt,Ukraine and Europe Project United Front Ahead ...,https://www.nytimes.com/2025/08/10/world/europ...,69
132,2025-08-10-23-18-32 +0000,nypost,Intel CEO Lip-Bu Tan to visit White House on M...,https://nypost.com/2025/08/10/business/intel-c...,69


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
147,81,2025-08-10-20-57-00 +0000,wsj,Intel CEO Lip-Bu Tan is set to visit the White...,https://www.wsj.com/tech/intel-ceo-singled-out...
185,60,2025-08-10-16-41-54 +0000,nyt,Dmitri Kozak Was a Key Putin Aide. He Lost Pow...,https://www.nytimes.com/2025/08/10/world/europ...
73,47,2025-08-11-08-58-23 +0000,bbc,Five Al Jazeera journalists killed in Israeli ...,https://www.bbc.com/news/articles/ceqyyrp3yq9o...
124,35,2025-08-11-00-09-23 +0000,nypost,"Nvidia, AMD to pay 15% of China chip sale reve...",https://nypost.com/2025/08/10/business/nvidia-...
49,33,2025-08-11-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Trump-Putin summit ...,https://nypost.com/2025/08/11/us-news/ny-postc...
22,32,2025-08-11-11-43-27 +0000,bbc,Colombian Senator Miguel Uribe dies two months...,https://www.bbc.com/news/articles/c78mmj97ygmo...
159,31,2025-08-10-20-08-00 +0000,wsj,Israel’s New War Plan Includes Fighting in Gaz...,https://www.wsj.com/world/middle-east/israels-...
17,29,2025-08-11-12-00-00 +0000,startribune,Yuen: She never got her quinceañera. So friend...,https://www.startribune.com/quinceanera-cincue...
106,26,2025-08-11-02-31-24 +0000,nypost,Girlfriend of US tourist shot dead during Puer...,https://nypost.com/2025/08/10/us-news/girlfrie...
96,26,2025-08-11-04-49-28 +0000,nypost,US military preparing to surge National Guard ...,https://nypost.com/2025/08/11/us-news/us-milit...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
